In [1]:
import os
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from urllib.parse import urljoin
import pickle
import re

In [ ]:
load_dotenv()

In [ ]:
DRIVER_PATH=os.getenv("DRIVER_PATH")
CHROM_PATH=os.getenv("CHROM_PATH")

In [7]:
def selenium_set():

    global driver
    
    # chrome path setting
    driver_path = DRIVER_PATH
    chrome_path = CHROM_PATH
    
    chrome_options = webdriver.ChromeOptions()
    
    chrome_options.binary_location = chrome_path
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    service = Service(executable_path=driver_path)
    
    driver = webdriver.Chrome(service=service, options=chrome_options)

In [8]:
#topic별 url 리스트 반환
def make_topic_urllist(data, base_url):
    urllist=[]
    topic_txt_lnb=pd.read_csv(data, encoding='utf-8')
    for row in topic_txt_lnb.iloc[:,1]:
        url = base_url + str(row)
        urllist.append(url) 
    return urllist

In [9]:
def scroll_click_extract(url):

    """
    - 페이지 바닥까지 스크롤하면서 '기사 더보기' 버튼을 반복 클릭
    - 반복이 끝나면 현재 페이지의 모든 기사 링크를 수집하여 DataFrame 반환
    """
    
    global driver
    driver.get(url)

    try:
        response = requests.get(url)
        response.raise_for_status()  # 문제가 발생하면 예외 발생
        html = response.text
    except requests.exceptions.RequestException as e:
        print(f"웹 페이지 요청 오류: {e}")
        exit()

    
    # XPath로 '기사 더보기' 버튼 찾기
    refresh_btn = driver.find_element(By.XPATH, '//a[contains(@class, "_CONTENT_LIST_LOAD_MORE_BUTTON")]')

    count = 0
    
    while True: ############ 클릭 끝까지로 바꾸기 ############
        count += 1
        print(f"[LOOP] {count} 번째 반복 중...")
        
        try:
            # 페이지 끝까지 스크롤하기
            body = driver.find_element(By.CSS_SELECTOR, 'body')
            for _ in range(10):  # 10번 정도 END 키 누르기
                body.send_keys(Keys.END)

            # 클릭
            # driver.execute_script("arguments[0].click();", refresh_btn)
            # time.sleep(1)  # 새 기사 로딩 대기

            # 최대 5초간 '기사 더보기' 버튼이 클릭 가능해질 때까지 기다림
            button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, '//a[contains(@class, "_CONTENT_LIST_LOAD_MORE_BUTTON")]'))
            )
            button.click()
            print(f" → 버튼 클릭 성공 ({count}회)")
                        
        except Exception as e:
            print("기사 더보기 버튼 클릭 실패:", e)
            break

    """
    --- 불러온 HTMl에서 기사 링크 모두 추출 ---
    """

    # 버튼 클릭으로 새로 로드된 기사들은 Selenium이 로드한 DOM에만 존재하므로 request에는 안잡힘.
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 뉴스 기사 링크 추출
    news_tag = soup.select('a.sa_text_title._NLOG_IMPRESSION')

    article_url_list = []

    for i in news_tag:
        href = i.get('href')
        full_url = urljoin(url, href)
        
        if href:
            article_url_list.append(full_url)

        else:
            break
    
    print("----url추출 및 list에 보냄----")

    return article_url_list

In [10]:
def crawl_with_selenium():

    global driver
    global data, base_url
    
    # __main__
    topic_urls = make_topic_urllist(data, base_url)
    
    article_url = [] # 모든 단일기사 url 저장 리스트

    for u in tqdm(range(len(topic_urls))):
        
        print("--토픽별 단일 기사 url 크롤링 시작---")
        article_url.append(scroll_click_extract(topic_urls[u]))
        
    return article_url

In [11]:
# selenium 실행 (단일 기사 url 추출)

driver = None
data = "../public_crawling_base_data/naver_topicset.csv"
base_url = "https://news.naver.com/breakingnews/section"
article_topic_df=[]
article_single_url_set=[]

def exec_code():
    selenium_set() 
    global article_topic_df
    global article_single_url_set
    
    article_topic_df=make_topic_urllist(data, base_url)
    article_single_url_set=crawl_with_selenium()

    return article_topic_df, article_single_url_set

In [9]:
exec_code()

  0%|          | 0/48 [00:00<?, ?it/s]

--토픽별 단일 기사 url 크롤링 시작---
[LOOP] 1 번째 반복 중...
 → 버튼 클릭 성공 (1회)
[LOOP] 2 번째 반복 중...
 → 버튼 클릭 성공 (2회)
[LOOP] 3 번째 반복 중...
 → 버튼 클릭 성공 (3회)
[LOOP] 4 번째 반복 중...
 → 버튼 클릭 성공 (4회)
[LOOP] 5 번째 반복 중...
 → 버튼 클릭 성공 (5회)
[LOOP] 6 번째 반복 중...
 → 버튼 클릭 성공 (6회)
[LOOP] 7 번째 반복 중...
 → 버튼 클릭 성공 (7회)
[LOOP] 8 번째 반복 중...
 → 버튼 클릭 성공 (8회)
[LOOP] 9 번째 반복 중...
 → 버튼 클릭 성공 (9회)
[LOOP] 10 번째 반복 중...
 → 버튼 클릭 성공 (10회)
[LOOP] 11 번째 반복 중...
 → 버튼 클릭 성공 (11회)
[LOOP] 12 번째 반복 중...
 → 버튼 클릭 성공 (12회)
[LOOP] 13 번째 반복 중...
 → 버튼 클릭 성공 (13회)
[LOOP] 14 번째 반복 중...
 → 버튼 클릭 성공 (14회)
[LOOP] 15 번째 반복 중...
 → 버튼 클릭 성공 (15회)
[LOOP] 16 번째 반복 중...
 → 버튼 클릭 성공 (16회)
[LOOP] 17 번째 반복 중...
 → 버튼 클릭 성공 (17회)
[LOOP] 18 번째 반복 중...
 → 버튼 클릭 성공 (18회)
[LOOP] 19 번째 반복 중...
 → 버튼 클릭 성공 (19회)
[LOOP] 20 번째 반복 중...
 → 버튼 클릭 성공 (20회)
[LOOP] 21 번째 반복 중...
 → 버튼 클릭 성공 (21회)
[LOOP] 22 번째 반복 중...
 → 버튼 클릭 성공 (22회)
[LOOP] 23 번째 반복 중...
 → 버튼 클릭 성공 (23회)
[LOOP] 24 번째 반복 중...
 → 버튼 클릭 성공 (24회)
[LOOP] 25 번째 반복 중...
 → 버튼 클릭 성공 (25회)
[LOOP] 26 번째 반복 중

  2%|▏         | 1/48 [00:55<43:31, 55.56s/it]

----url추출 및 list에 보냄----
--토픽별 단일 기사 url 크롤링 시작---
[LOOP] 1 번째 반복 중...
 → 버튼 클릭 성공 (1회)
[LOOP] 2 번째 반복 중...
 → 버튼 클릭 성공 (2회)
[LOOP] 3 번째 반복 중...
 → 버튼 클릭 성공 (3회)
[LOOP] 4 번째 반복 중...
 → 버튼 클릭 성공 (4회)
[LOOP] 5 번째 반복 중...
 → 버튼 클릭 성공 (5회)
[LOOP] 6 번째 반복 중...
 → 버튼 클릭 성공 (6회)
[LOOP] 7 번째 반복 중...
 → 버튼 클릭 성공 (7회)
[LOOP] 8 번째 반복 중...
 → 버튼 클릭 성공 (8회)
[LOOP] 9 번째 반복 중...
 → 버튼 클릭 성공 (9회)
[LOOP] 10 번째 반복 중...
 → 버튼 클릭 성공 (10회)
[LOOP] 11 번째 반복 중...
 → 버튼 클릭 성공 (11회)
[LOOP] 12 번째 반복 중...
 → 버튼 클릭 성공 (12회)
[LOOP] 13 번째 반복 중...
 → 버튼 클릭 성공 (13회)
[LOOP] 14 번째 반복 중...
 → 버튼 클릭 성공 (14회)
[LOOP] 15 번째 반복 중...
 → 버튼 클릭 성공 (15회)
[LOOP] 16 번째 반복 중...
 → 버튼 클릭 성공 (16회)
[LOOP] 17 번째 반복 중...
 → 버튼 클릭 성공 (17회)
[LOOP] 18 번째 반복 중...
 → 버튼 클릭 성공 (18회)
[LOOP] 19 번째 반복 중...
 → 버튼 클릭 성공 (19회)
[LOOP] 20 번째 반복 중...
 → 버튼 클릭 성공 (20회)
[LOOP] 21 번째 반복 중...
 → 버튼 클릭 성공 (21회)
[LOOP] 22 번째 반복 중...
 → 버튼 클릭 성공 (22회)
[LOOP] 23 번째 반복 중...
 → 버튼 클릭 성공 (23회)
[LOOP] 24 번째 반복 중...
 → 버튼 클릭 성공 (24회)
[LOOP] 25 번째 반복 중...
 → 버튼 클릭 성

  4%|▍         | 2/48 [08:26<3:40:58, 288.23s/it]

...

 98%|█████████▊| 47/48 [1:39:40<00:35, 35.01s/it]

----url추출 및 list에 보냄----
--토픽별 단일 기사 url 크롤링 시작---
[LOOP] 1 번째 반복 중...
 → 버튼 클릭 성공 (1회)
[LOOP] 2 번째 반복 중...
 → 버튼 클릭 성공 (2회)
[LOOP] 3 번째 반복 중...
 → 버튼 클릭 성공 (3회)
[LOOP] 4 번째 반복 중...
 → 버튼 클릭 성공 (4회)
[LOOP] 5 번째 반복 중...
 → 버튼 클릭 성공 (5회)
[LOOP] 6 번째 반복 중...
 → 버튼 클릭 성공 (6회)
[LOOP] 7 번째 반복 중...
 → 버튼 클릭 성공 (7회)
[LOOP] 8 번째 반복 중...
 → 버튼 클릭 성공 (8회)
[LOOP] 9 번째 반복 중...
 → 버튼 클릭 성공 (9회)
[LOOP] 10 번째 반복 중...
 → 버튼 클릭 성공 (10회)
[LOOP] 11 번째 반복 중...
 → 버튼 클릭 성공 (11회)
[LOOP] 12 번째 반복 중...
 → 버튼 클릭 성공 (12회)
[LOOP] 13 번째 반복 중...
 → 버튼 클릭 성공 (13회)
[LOOP] 14 번째 반복 중...
 → 버튼 클릭 성공 (14회)
[LOOP] 15 번째 반복 중...
 → 버튼 클릭 성공 (15회)
[LOOP] 16 번째 반복 중...
 → 버튼 클릭 성공 (16회)
[LOOP] 17 번째 반복 중...
 → 버튼 클릭 성공 (17회)
[LOOP] 18 번째 반복 중...
 → 버튼 클릭 성공 (18회)
[LOOP] 19 번째 반복 중...
 → 버튼 클릭 성공 (19회)
[LOOP] 20 번째 반복 중...
 → 버튼 클릭 성공 (20회)
[LOOP] 21 번째 반복 중...
 → 버튼 클릭 성공 (21회)
[LOOP] 22 번째 반복 중...
 → 버튼 클릭 성공 (22회)
[LOOP] 23 번째 반복 중...
 → 버튼 클릭 성공 (23회)
[LOOP] 24 번째 반복 중...
 → 버튼 클릭 성공 (24회)
[LOOP] 25 번째 반복 중...
 → 버튼 클릭 성

100%|██████████| 48/48 [1:42:33<00:00, 128.19s/it]

----url추출 및 list에 보냄----


(['https://news.naver.com/breakingnews/section/100/264',
  'https://news.naver.com/breakingnews/section/100/265',
  'https://news.naver.com/breakingnews/section/100/268',
  'https://news.naver.com/breakingnews/section/100/266',
  'https://news.naver.com/breakingnews/section/100/267',
  'https://news.naver.com/breakingnews/section/100/269',
  'https://news.naver.com/breakingnews/section/101/259',
  'https://news.naver.com/breakingnews/section/101/258',
  'https://news.naver.com/breakingnews/section/101/261',
  'https://news.naver.com/breakingnews/section/101/771',
  'https://news.naver.com/breakingnews/section/101/260',
  'https://news.naver.com/breakingnews/section/101/262',
  'https://news.naver.com/breakingnews/section/101/310',
  'https://news.naver.com/breakingnews/section/101/263',
  'https://news.naver.com/breakingnews/section/102/249',
  'https://news.naver.com/breakingnews/section/102/250',
  'https://news.naver.com/breakingnews/section/102/251',
  'https://news.naver.com/break

In [21]:
# article_single_url_set을 피클 백업파일로 저장
with open('../crawling_base_data/url_list_backup.pkl', 'wb') as f:
    pickle.dump(article_single_url_set, f)

In [29]:
# 컬럼을 토픽명으로 달아준 데이터프레임으로 만든다음 피클로 저장하는 셀

col_temp = pd.read_csv(data)
name_list = col_temp['name'].tolist()

article_final_url_df_to_pickle = []

for i in range(len(article_single_url_set)):
    temp_df = pd.DataFrame(article_single_url_set[i], columns=[name_list[i]])
    article_final_url_df_to_pickle.append(temp_df)


with open('../crawling_base_data/final_url_df_backup_.pkl', 'wb') as f:
    pickle.dump(article_final_url_df_to_pickle, f)   

In [17]:
# 기사 하나의 내용 크롤링 (기사 제목, 기사 내용)
def article_crawler(url):
    html=None
    try:
        response = requests.get(url)
        response.raise_for_status()  # 문제가 발생하면 예외 발생
        html = response.text
        
    except requests.exceptions.RequestException as e:
        print(f"웹 페이지 요청 오류: {e}")
        pass
        
    if html is None:
        return None
        
    soup=BeautifulSoup(html,'html.parser')

    # copyright 태그가 있는지 체크후 넘겨주기
    c_tag = soup.find('p', {'class': 'c_text'})
    if not c_tag:
        return None
    copyright = c_tag.text.strip()
    
    # Copyright Issue: content 내의 Copyright 내역 검색하여 [AI, ...] 키워드로 기사 Ban
    AI_ban=['AI', '인공', '학습', '크롤링', '상업', '수익']
            
    for i in AI_ban:
        if i in copyright:
            return None

    # title 태그 체크
    t_tag = soup.find('title')
    if not t_tag:
        return None
    title = t_tag.text.strip()

    # content 태그 체크
    a_tag = soup.find("article", id='dic_area')
    if not a_tag:
        return None
    content = a_tag.text.strip()
    
    return title, content

In [23]:
# 위 셀에서 크롤링한 데이터를 데이터프레임으로 만들고 Pkl파일로 저장
article_contents = []

total = sum(len(df) for df in article_final_url_df_to_pickle)

with tqdm(total=total, desc="전체 URL 크롤링", ncols=0) as pbar:
    for i in range(len(article_final_url_df_to_pickle)):
        temp_df = pd.DataFrame(columns=["title", "content"])

        for j in range(len(article_final_url_df_to_pickle[i])):
            # copyright에서 걸러진 기사는 패스
            contents = article_crawler(article_final_url_df_to_pickle[i].iloc[j, 0])
            if contents is not None:
                title, content = contents
                row_df = pd.DataFrame([(title, content)], columns=["title", "content"])
                temp_df = pd.concat([temp_df, row_df], axis=0, ignore_index=True)
            else:
                continue
            pbar.update(1)
        article_contents.append(temp_df)
        with open(f'../crawling_base_data/backup_files/article_contents_backup_df_{i}.pkl', 'wb') as f:
            pickle.dump(article_contents[i], f) 
        print(f'saved : article_contents_backup_df_{i}.pkl')
        

전체 URL 크롤링:   1% 832/79818 [05:35<7:08:09,  3.07it/s] 

saved : article_contents_backup_df_0.pkl


전체 URL 크롤링:   6% 5187/79818 [43:04<6:29:34,  3.19it/s] 

saved : article_contents_backup_df_1.pkl


전체 URL 크롤링:   7% 5465/79818 [44:57<13:21:31,  1.55it/s]

...

saved : article_contents_backup_df_45.pkl


전체 URL 크롤링:  59% 46742/79818 [4:52:18<2:56:58,  3.11it/s]

saved : article_contents_backup_df_46.pkl


전체 URL 크롤링:  62% 49426/79818 [5:09:27<3:10:17,  2.66it/s] 

saved : article_contents_backup_df_47.pkl
